# Sberbank Russian Housing Market.
## Part 2. Data preparation and creating a model. 

*Olga Voitka*

This notebook shows my work on Kaggle competition. The goal of this competition is to predict the sale price of a given property based on the data provided by Sberbank (Russia’s bank). This the second part where I do data engineering and create an XGBoost model. 

To get this results I've tried:
- using gbm and xgboost models;
- using several technics to deal with missing data: missForest and mice packages, replacing missing data with means;
- creating additional variables;
- adding variables from macro data set; 
- choosing variables for a final model with correlated matrix, field's variances and relative influence.  

## 1. Data Engineering. 

We have 3 data files: train.csv, test.csv.
Let's upload them.

In [1]:
library(data.table)
library(Matrix)
library(xgboost)
library(caret)
library(dplyr)

# Load CSV files
cat("Read data")
train <- fread('train.csv', na.strings = "NA")
test <- fread("test.csv", sep=",", na.strings = "NA")

Warning message:
"package 'xgboost' was built under R version 3.3.3"Warning message:
"package 'caret' was built under R version 3.3.3"Loading required package: lattice
Loading required package: ggplot2
------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: 'dplyr'

The following object is masked from 'package:xgboost':

    slice

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Read 30471 rows and 292 (of 292) columns from 0.043 GB file in 00:00:04


In [2]:
# Resample train data
train = train[price_doc>1000000 & price_doc<111111112,]


# Transform target variable, so that we can use RMSE in XGBoost
train$price_doc <- log1p(as.integer(train$price_doc))

# Replace NAs with -1 
train[is.na(train)] <- -1
test[is.na(test)] <- -1

# Combine data
data <- rbind(train, test, fill=TRUE)
str(data)
sample(data, 10)

Classes 'data.table' and 'data.frame':	37152 obs. of  292 variables:
 $ id                                   : int  1 2 3 4 5 6 7 8 9 10 ...
 $ timestamp                            : chr  "2011-08-20" "2011-08-23" "2011-08-27" "2011-09-01" ...
 $ full_sq                              : num  43 34 43 89 77 67 25 44 42 36 ...
 $ life_sq                              : num  27 19 29 50 77 46 14 44 27 21 ...
 $ floor                                : num  4 3 2 9 4 14 10 5 5 9 ...
 $ max_floor                            : num  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
 $ material                             : num  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
 $ build_year                           : num  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
 $ num_room                             : num  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
 $ kitch_sq                             : num  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
 $ state                                : num  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
 $ product_type                    

cafe_count_1500_price_high,exhibition_km,cafe_count_5000_price_4000,thermal_power_plant_raion,church_count_3000,nuclear_reactor_km,ID_big_road1,cafe_count_1500,cafe_count_3000_price_1000,market_count_1500
0,7.0237049,4,no,4,5.718519,1,34,22,1
0,2.3588405,3,no,7,3.489954,2,17,11,0
0,4.9582143,3,no,11,7.506612,3,14,17,5
0,5.0296963,2,no,2,9.522538,1,23,14,2
1,1.3396523,108,no,121,8.671016,4,272,267,2
0,2.5534235,14,yes,12,8.757686,4,14,37,0
0,3.3733429,16,no,29,11.807532,4,44,57,1
0,7.7189668,4,no,4,5.332926,1,29,15,1
0,2.4494532,8,no,8,1.652275,5,15,28,2
0,0.7118749,13,no,9,7.353920,6,9,15,0


In [3]:
# If full_sq is smaller than life_sq, clean full_sq and life_sq. 
data %>% select(full_sq, life_sq) %>% filter(life_sq ==-1) %>% mutate(life_sq = full_sq)
data %>% select(full_sq, life_sq) %>% filter(full_sq < life_sq) %>% mutate(full_sq = NA, life_sq = NA)

# Build_year
data$build_year <- as.character(data$build_year)
data$build_year[data$build_year=="0"] <- -1
data$build_year[data$build_year=="1"] <- -1
data$build_year[data$build_year=="2"] <- -1
data$build_year[data$build_year=="3"] <- -1
data$build_year[data$build_year=="20"] <- "1920"
data$build_year[data$build_year=="71"] <- "1971"
data$build_year[data$build_year=="215"] <- "2015"
data$build_year[data$build_year=="4965"] <- "1965"
data$build_year[data$build_year=="20052009"] <- "2007"

data$build_year <- as.integer(data$build_year)

#Let's see if all max_floor > floor and replace odd values with NA
data %>% select(max_floor, floor) %>% filter(max_floor < floor) %>% mutate(max_floor = -1, floor = -1)


#num_rooms
data$num_room <- ifelse(data$num_room==0, -1, data$num_room)
data$num_room <- as.integer(data$num_room)

#state
data$state <- ifelse(data$state==33, 3, data$state)

full_sq,life_sq
73,73
110,110
167,167
53,53
81,81
166,166
167,167
123,123
58,58
155,155


full_sq,life_sq
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA


max_floor,floor
-1,-1
-1,-1
-1,-1
-1,-1
-1,-1
-1,-1
-1,-1
-1,-1
-1,-1
-1,-1


In [4]:
# Convert characters to factors/numeric
data$thermal_power_plant_raion <- ifelse(data$thermal_power_plant_raion=="no",0,1) 
data$incineration_raion <- ifelse(data$incineration_raion=="no",0,1)
data$oil_chemistry_raion <- ifelse(data$oil_chemistry_raion=="no",0,1)
data$radiation_raion <- ifelse(data$radiation_raion=="no",0,1)
data$railroad_terminal_raion <- ifelse(data$railroad_terminal_raion=="no",0,1)
data$big_market_raion <- ifelse(data$big_market_raion=="no",0,1)
data$nuclear_reactor_raion <- ifelse(data$nuclear_reactor_raion=="no",0,1)
data$detention_facility_raion <- ifelse(data$detention_facility_raion=="no",0,1)
data$culture_objects_top_25 <- ifelse(data$culture_objects_top_25=="no",0,1)
data$water_1line <- ifelse(data$water_1line=="no",0,1)
data$big_road1_1line <- ifelse(data$big_road1_1line=="no",0,1)
data$railroad_1line <- ifelse(data$railroad_1line=="no",0,1)


# Work with timestamp
data$timestamp <- as.Date(as.character(data$timestamp))
data$year <- as.integer(format(data$timestamp, "%Y"))

# New features
data$rel_floor = data$floor/data$max_floor
data$diff_floor = data$max_floor-data$floor
data$rel_kitchen_sq = data$kitch_sq/data$full_sq
data$rel_life_sq = data$life_sq/data$full_sq
data$rel_kitchen_life = data$kitch_sq/data$life_sq
data$rel_sq_per_floor = data$full_sq/data$floor
data$diff_life_sq = data$full_sq-data$life_sq
data$building_age = data$year - data$build_year
data$state.material <- data$state*data$material

In [5]:
# woork with sub_area, create new variables neighborhood
Zelenograd <- c("Krjukovo","Matushkino","Savelki","Silino","Staroe Krjukovo")
Novomoskovsky <- c("Poselenie Desjonovskoe","Poselenie Filimonkovskoe","Poselenie Kokoshkino","Poselenie Marushkinskoe","Poselenie Moskovskij","Poselenie Mosrentgen","Poselenie Rjazanovskoe","Poselenie Shherbinka","Poselenie Sosenskoe","Poselenie Vnukovskoe","Poselenie Voskresenskoe")
Troitsky <- c("Poselenie Kievskij","Poselenie Klenovskoe","Poselenie Krasnopahorskoe","Poselenie Mihajlovo-Jarcevskoe","Poselenie Novofedorovskoe","Poselenie Pervomajskoe","Poselenie Rogovskoe","Poselenie Shhapovskoe","Poselenie Voronovskoe","Troickij okrug")
Northern <- c("Ajeroport","Begovoe","Beskudnikovskoe","Dmitrovskoe","Golovinskoe","Horoshevskoe","Hovrino","Koptevo","Levoberezhnoe","Molzhaninovskoe","Savelovskoe","Sokol","Timirjazevskoe","Vojkovskoe","Vostochnoe Degunino","Zapadnoe Degunino")
Southwest <- c("Akademicheskoe","Cheremushki","Gagarinskoe","Jasenevo","Juzhnoe Butovo","Kon'kovo","Kotlovka","Lomonosovskoe","Obruchevskoe","Severnoe Butovo","Teplyj Stan","Zjuzino")
Northeast <- c("Alekseevskoe","Altuf'evskoe","Babushkinskoe","Bibirevo","Butyrskoe","Jaroslavskoe","Juzhnoe Medvedkovo","Lianozovo","Losinoostrovskoe","Mar'ina Roshha","Marfino","Ostankinskoe","Otradnoe","Rostokino","Severnoe","Severnoe Medvedkovo","Sviblovo")
Central <- c("Arbat","Basmannoe","Hamovniki","Jakimanka","Krasnosel'skoe","Meshhanskoe","Presnenskoe","Taganskoe","Tverskoe","Zamoskvorech'e")
Southern <- c("Birjulevo Vostochnoe","Birjulevo Zapadnoe","Brateevo","Caricyno","Chertanovo Central'noe","Chertanovo Juzhnoe","Chertanovo Severnoe","Danilovskoe","Donskoe","Moskvorech'e-Saburovo","Nagatino-Sadovniki","Nagatinskij Zaton","Nagornoe","Orehovo-Borisovo Juzhnoe","Orehovo-Borisovo Severnoe","Zjablikovo")
Eastern <- c("Bogorodskoe","Gol'janovo","Ivanovskoe","Izmajlovo","Kosino-Uhtomskoe","Metrogorodok","Novogireevo","Novokosino","Perovo","Preobrazhenskoe","Severnoe Izmajlovo","Sokol'niki","Sokolinaja Gora","Veshnjaki","Vostochnoe","Vostochnoe Izmajlovo")
Western <-c ("Dorogomilovo","Filevskij Park","Fili Davydkovo","Krylatskoe","Kuncevo","Mozhajskoe","Novo-Peredelkino","Ochakovo-Matveevskoe","Prospekt Vernadskogo","Ramenki","Solncevo","Troparevo-Nikulino","Vnukovo")
Northwest <- c("Horoshevo-Mnevniki","Juzhnoe Tushino","Kurkino","Mitino","Pokrovskoe Streshnevo","Severnoe Tushino","Shhukino","Strogino")
Southeast <- c("Juzhnoportovoe","Kapotnja","Kuz'minki","Lefortovo","Ljublino","Mar'ino","Nekrasovka","Nizhegorodskoe","Pechatniki","Rjazanskij","Tekstil'shhiki","Vyhino-Zhulebino")

data$neighborhood[data$sub_area %in% Novomoskovsky] <- 1 #"Novomoskovsky"
data$neighborhood[data$sub_area %in% Troitsky] <- 2 #"Troitsky"
data$neighborhood[data$sub_area %in% Northern] <- 3 #"Northern"
data$neighborhood[data$sub_area %in% Southwest] <- 4 #"Southwest"
data$neighborhood[data$sub_area %in% Northeast] <- 5 #"Northeast"
data$neighborhood[data$sub_area %in% Central] <- 6 #"Central"
data$neighborhood[data$sub_area %in% Southern] <- 7 #"Southern"
data$neighborhood[data$sub_area %in% Eastern] <- 8 #"Eastern"
data$neighborhood[data$sub_area %in% Western] <- 9 #"Western"
data$neighborhood[data$sub_area %in% Northwest] <- 10 #"Northwest"
data$neighborhood[data$sub_area %in% Southeast] <- 11 #"Southeast"
data$neighborhood[data$sub_area %in% Zelenograd] <- 12 #"Zelenograd"
data$neighborhood<-as.factor(data$neighborhood)


In [6]:
# Coding categorycal features
features = colnames(data)
for (f in features){
  if( (class(data[[f]]) == "character") || (class(data[[f]]) == "factor"))
  {
    levels = unique(data[[f]])
    data[[f]] = as.numeric(factor(data[[f]], level = levels)) 
  }
}

data$material <- as.factor(data$material)
data$build_year <- as.factor(data$build_year)
data$state <- as.factor(data$state)
data$product_type <- as.factor(data$product_type)
data$sub_area <- as.factor(data$sub_area)
data$ecology <- as.factor(data$ecology)

In [7]:
# Encoding categorycal features to dummies variables
data <- as.data.frame(data)
ohe_feats <- c('material', 'build_year', 'state', 'product_type', 'sub_area', 'ecology', 'neighborhood')

In [8]:
dummies <- dummyVars(~ material + build_year + state + product_type + sub_area + ecology + neighborhood, data = data)
df_all_ohe <- as.data.frame(predict(dummies, newdata = data))
df_all_combined <- cbind(data[,-c(which(colnames(data) %in% ohe_feats))],df_all_ohe)
data <- as.data.table(df_all_combined)

In [9]:
# Delete uninportant features 
data[,c("year", "timestamp", "young_male", "school_education_centers_top_20_raion", "0_17_female", "railroad_1line", "7_14_female", "0_17_all", "children_school",
        "16_29_male", "mosque_count_3000", "female_f", "church_count_1000", "railroad_terminal_raion",
        "mosque_count_5000", "big_road1_1line", "mosque_count_1000", "7_14_male", "0_6_female", "oil_chemistry_raion",
        "young_all", "0_17_male", "ID_bus_terminal", "university_top_20_raion", "mosque_count_500","ID_big_road1",
        "ID_railroad_terminal", "ID_railroad_station_walk", "ID_big_road2", "ID_metro", "ID_railroad_station_avto",
        "0_13_all", "mosque_count_2000", "work_male", "16_29_all", "young_female", "work_female", "0_13_female",
        "ekder_female", "7_14_all", "big_church_count_500",
        "leisure_count_500", "cafe_sum_1500_max_price_avg", "leisure_count_2000",
        "office_count_500", "male_f", "nuclear_reactor_raion", "0_6_male", "church_count_500", "build_count_before_1920",
        "thermal_power_plant_raion", "cafe_count_2000_na_price", "cafe_count_500_price_high",
        "market_count_2000", "trc_count_500", "market_count_1000", "work_all", "additional_education_raion",
        "build_count_slag", "leisure_count_1000", "0_13_male", "office_raion",
        "raion_build_count_with_builddate_info", "market_count_3000", "ekder_all", "trc_count_1000", "build_count_1946-1970",
        "office_count_1500", "cafe_count_1500_na_price", "big_church_count_5000", "big_church_count_1000", "build_count_foam",
        "church_count_1500", "church_count_3000", "leisure_count_1500",
        "16_29_female", "build_count_after_1995", "cafe_avg_price_1500", "office_sqm_1000", "cafe_avg_price_5000", "cafe_avg_price_2000",
        "big_church_count_1500", "full_all", "cafe_sum_5000_min_price_avg",
        "office_sqm_2000", "church_count_5000","0_6_all", "detention_facility_raion", "cafe_avg_price_3000")] <- NULL

varnames <- setdiff(colnames(data), c("id","price_doc"))

## Create XGB model and make a prediction. 

Next step is preparing data sets for using in XGB model. 

In [10]:
# Create sparse matrix
train_sparse <- Matrix(as.matrix(sapply(data[price_doc > -1,varnames,with=FALSE],as.numeric)), sparse=TRUE)
test_sparse <- Matrix(as.matrix(sapply(data[is.na(price_doc),varnames,with=FALSE],as.numeric)), sparse=TRUE)
y_train <- data[!is.na(price_doc),price_doc]
test_ids <- data[is.na(price_doc),id]
dtrain <- xgb.DMatrix(data=train_sparse, label=y_train)
dtest <- xgb.DMatrix(data=test_sparse);

gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1718838,91.8,13897102,742.2,15868533,847.5
Vcells,63361573,483.5,174735652,1333.2,218412664,1666.4


In [11]:
# Find the best parameters to run a model
best_param = list()
best_seednumber = 1234
best_rmse=1000000
best_rmse_index=0

for (iter in 1:3) {
  param <- list(objective = "reg:linear",
                eval_metric = "rmse",
                booster = "gbtree",
                max_depth = sample(4:10, 1),
                eta = runif(1, .01, .3),
                gamma = runif(1, 0.0, 0.2),
                subsample = runif(1, .6, .9),
                colsample_bytree = runif(1, .5, .8),
                min_child_weight = sample(1:40, 1),
                max_delta_step = sample(1:10, 1)
  )
  cv.nround = 150
  cv.nfold = 5
  seed.number = sample.int(10000, 1)[[1]]
  set.seed(seed.number)
  mdcv <- xgb.cv(dtrain, params = param, nthread=6, nfold=cv.nfold, 
                 nrounds=cv.nround, verbose = F, early_stopping_rounds=4, maximize=FALSE, missing = -1)
  print(mdcv)
  min_rmse = min(mdcv$evaluation_log$test_rmse_mean)
  min_rmse_index = which.min(mdcv$evaluation_log$test_rmse_mean)
  print(iter)
  if (min_rmse < best_rmse) {
    best_rmse = min_rmse
    best_rmse_index = min_rmse_index
    best_seednumber = seed.number
    best_param = param
  }
}

best_rmse
best_rmse_index
best_seednumber
best_param

##### xgb.cv 5-folds
    iter train_rmse_mean train_rmse_std test_rmse_mean test_rmse_std
       1      15.0473472    0.001421598     15.0473462   0.005686408
       2      14.9160102    0.001421589     14.9160094   0.005686915
       3      14.7846760    0.001421671     14.7846750   0.005688026
       4      14.6533424    0.001422075     14.6533410   0.005689136
       5      14.5220100    0.001422369     14.5220090   0.005689696
---                                                                 
     146       0.2725346    0.002431972      0.3326486   0.004298976
     147       0.2718052    0.002481053      0.3326144   0.004307568
     148       0.2712020    0.002562475      0.3325818   0.004319258
     149       0.2703220    0.002346623      0.3324194   0.004339058
     150       0.2698398    0.002559907      0.3323752   0.004390752
Best iteration:
 iter train_rmse_mean train_rmse_std test_rmse_mean test_rmse_std
  150       0.2698398    0.002559907      0.3323752   0.004390752
[1]

[1] 0.3314044

[1] 111

[1] 5480

$objective
[1] "reg:linear"

$eval_metric
[1] "rmse"

$booster
[1] "gbtree"

$max_depth
[1] 5

$eta
[1] 0.1719334

$gamma
[1] 0.009022116

$subsample
[1] 0.8791967

$colsample_bytree
[1] 0.7594408

$min_child_weight
[1] 31

$max_delta_step
[1] 2

In [14]:
# Create model with best parameters
xgb = xgboost(dtrain, params=best_param, 
              nrounds=150, nthread=2, p_seed=best_seednumber, missing = -1, verbose = FALSE)

In [15]:
# Create predictions
pred1 <- round(predict(xgb, dtest, missing = -1), 4)

# Write predictions into file
preds2 <- exp(pred1)


preds = tibble(id = test$id, price_doc = preds2)
write.csv(preds, "preds66.csv", row.names = FALSE)

This model gives a RMSE = 0.31994, what ranked me top  

## Conclusion. 
In this project I've worked from RMSE equal 0.3828 to 0.3199. To get this result I've used different methods for data exploration, feature engineering and machine learning algorithms. It was an interesting and challenging task to find a way to deal with dirty data and big dimensionality.  